In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from datetime import date
import pandas as pd
import time

In [2]:
path = 'ImagesAttendance'
images = []
classNames = []
myList = os.listdir(path)
myList.pop(0)
print(myList)

['209006Josh.jpg', '209070Kobe.jpg', '209071Messi.jpg', '209072Ronaldo.jpg']


In [3]:
for cls in myList:
    currentImg = cv2.imread(f'{path}/{cls}')
    images.append(currentImg)
    classNames.append(os.path.splitext(cls)[0])

print(classNames)

['209006Josh', '209070Kobe', '209071Messi', '209072Ronaldo']


In [4]:
encodeList = []
for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)

    
encodeListKnown = encodeList
print("Encoding complete")

Encoding complete


In [5]:
def markAttendance(name):
    with open("attendance face recog.csv", 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dateString  = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dateString}')
            
   

In [6]:
def markMaster():
    df = pd.read_csv("attendance face recog.csv")
    df = df['Name'].str.extract('([^a-zA-Z]+)([a-zA-Z]+)', expand=True)
    df.columns = ['UID', 'Name']
    now = datetime.now()
    today = now.strftime("%d %B,%Y : %H:%M")
    df[today] = 'p'
    df = df.drop(['Name'], axis=1)
    df['UID'] = pd.to_numeric(df['UID'])
    master = pd.read_csv('master.csv')
    master['UID'] = pd.to_numeric(master['UID'])
    merge = pd.merge(master, df, on="UID", how = "outer")
    merge[today] = merge[today].fillna('a')
    merge.to_csv('master.csv', index = False)
    temp = pd.read_csv("attendance face recog.csv")
    df_empty = temp[0:0]
    df_empty.to_csv("attendance face recog.csv", index=False)

In [7]:
capture_duration = 20
cap = cv2.VideoCapture(0)

In [8]:
start_time = time.time()
while( int(time.time() - start_time) < capture_duration ):
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    facesCurrentFrame = face_recognition.face_locations(imgS)
    encodeCurrentFrame = face_recognition.face_encodings(imgS, facesCurrentFrame)
    
    for encodeFace, faceLoc in zip(encodeCurrentFrame, facesCurrentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDistance = face_recognition.face_distance(encodeListKnown, encodeFace)
        #print(faceDistance)
        matchIndex = np.argmin(faceDistance)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img, (x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img, (x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)
    
       
    
    
    
    cv2.imshow('Webcam', img)
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()
markMaster()

In [9]:
#temp = pd.read_csv("temp.csv")
#names = [x for x in temp.columns]
#df = pd.DataFrame(columns=names)

In [10]:
#df.to_csv("temp1.csv",index = False)

In [11]:
#df_empty = temp[0:0]
#df_empty.to_csv("empty", index=False)